# Two one dimensional quantum wells 

[Go back to index](./index.ipynb)

In this widget, one can choice the position, height and width of two quantum wells. The widget gives the eigenfunctions and eigenvalues from the numerical solution. 

In [13]:
%matplotlib widget

from numpy import linspace, sqrt, ones, arange, diag, argsort, zeros
from scipy.linalg import eigh_tridiagonal
import matplotlib.pyplot as plt

colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']
ixx = 0

def doublewell_potential(x, width1, depth1, width2, depth2, dist):
    xa = zeros(len(x))
    xb = zeros(len(x))
    
    for i in range(len(x)):
        if x[i] > -dist/2.0 - width1 and x[i] < -dist/2.0:
            xa[i] = depth1

    for i in range(len(x)):
        if x[i] > dist/2.0 and x[i] < dist/2.0 + width2:
            xb[i] = depth2
    
    return xa + xb
            
    
def diagonalisierung(hquer, L, N, pot = doublewell_potential, width1 = 0.5, depth1 = -0.2,
                     width2 = 0.5, depth2 = -0.2, dist = 1.0):
    """Calculated sorted eigenvalues and eigenfunctions. 

       Input:
         hquer: Planck constant
         L: set viewed interval [-L,L] 
         N: number of grid points i.e. size of the matrix 
         pot: potential function of the form pot
         x0: center of the quantum well
         width: the width of the quantum well
         depth: the depth of the quantum well
       Ouput:
         ew: sorted eigenvalues (array of length N)
         ef: sorted eigenfunctions, ef[:,i] (size N*N)
         x:  grid points (arry of length N)
         dx: grid space
         V:  Potential at positions x (array of length N)
    """
    x = linspace(-L, L, N+2)[1:N+1]               # grid points 
    dx = x[1] - x[0]                              # grid spacing
    V = pot(x, width1, depth1, width2, depth2, dist)
    z = hquer**2 /2.0/dx**2                       # second diagonals

    ew, ef = eigh_tridiagonal(V+2.0*z, -z*ones(N-1))    
    ew = ew.real                                  # real part of the eigenvalues
    ind = argsort(ew)                             # Indizes f. sort. Array
    ew = ew[ind]                                  # Sort the ew by ind
    ef = ef[:, ind]                               # Sort the columns

    ef = ef/sqrt(dx)                              # Correct standardization 
    return ew, ef, x, dx, V


def plot_eigenfunktionen(ax, ew, ef, x, V, width=1, Emax=0.1, fak=5.0, single = 0):
    """Plot of the lowest eigenfunctions 'ef' in the potential 'V (x)'
       at the level of the eigenvalues 'ew' in the plot area 'ax'.
    """
    
    fak = fak/100.0;
    
    ax[0].plot(x, V, c='r', linewidth=1.0)
    ax[1].plot(x, V, c='r', linewidth=1.0)
    
    ax[0].set_xlim([min(x), max(x)])
    ax[0].set_ylim([min(V)-0.05, Emax])
    
    ax[0].set_xlabel(r'$x/a$', fontsize = 10)
    ax[0].set_ylabel(r'$V(x)/V_0\ \rm{, Eigenfunctions\ with\ Eigenvalues}$', fontsize = 10)
    
    ax[1].set_xlim([min(x), max(x)])
    ax[1].set_ylim([min(V)-0.05, Emax])
    
    ax[1].set_xlabel(r'$x/a$', fontsize = 10)
    ax[1].set_ylabel(r'$V(x)/V_0\ \rm{, Eigenfunctions\ with\ Eigenvalues}$', fontsize = 10)

    ax[1].plot(x, fak*abs(ef[:, single])**2+ew[single], linewidth=width+.1, color=colors[single%len(colors)])
    
    indmax = sum(ew<=Emax)                         
    if not hasattr(width, "__iter__"):           
        width = width*ones(indmax)               
    for i in arange(indmax):                     
        ax[0].plot(x, fak*abs(ef[:, i])**2+ew[i], linewidth=width[i]+.1, color=colors[i%len(colors)])


In [14]:
from ipywidgets import FloatSlider, jslink, VBox, HBox

L = 1.5                                              # x-Bereich ist [-L,L]
N = 200                                              # Zahl der Gitterpunkte
hquer = 0.06                                         # effektives hquer
sigma_x = 0.1                                        # Breite Gauss
zeiten = linspace(0.0, 10.0, 400)                    # Zeiten f. Zeitentw.

style = {'description_width': 'initial'}

swidth1 = FloatSlider(value = 0.5, min = 0.1, max = 1.0, description = 'Width (1st well): ', style = style)
sdepth1 = FloatSlider(value = -0.2, min = -1.0, max = 0.0, description = 'Depth (1st well): ')

swidth2 = FloatSlider(value = 0.5, min = 0.1, max = 1.0, description = 'Width (2nd well): ', style = style)
sdepth2 = FloatSlider(value = -0.2, min = -1.0, max = 0.0, description = 'Depth (2nd well): ')

sdist = FloatSlider(value = 0.5, min = 0.0, max = L, description = r'Gap distance: ')
sfak = FloatSlider(value = 5, min = 1.0, max = 10.0, step = 1.0, description = r'Zoom factor: ')

ew, ef, x, dx, V = diagonalisierung(hquer, L, N)
    
fig, ax = plt.subplots(2, 1, figsize=(8, 8))

fig.suptitle('Numerial Solution of One Dimension Schrödinger Equation', fontsize = 10)
plot_eigenfunktionen(ax, ew, ef, x, V)

def on_width_change1(change):
    global ew, ef, x, dx, V
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, 
                                        width1 = swidth1.value, depth1 = sdepth1.value,
                                        width2 = swidth2.value, depth2 = sdepth2.value,
                                        dist = sdist.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)

def on_depth_change1(change):
    global ew, ef, x, dx, V
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, 
                                        width1 = swidth1.value, depth1 = sdepth1.value,
                                        width2 = swidth2.value, depth2 = sdepth2.value,
                                        dist = sdist.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)
    

def on_width_change2(change):
    global ew, ef, x, dx, V
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, 
                                        width1 = swidth1.value, depth1 = sdepth1.value,
                                        width2 = swidth2.value, depth2 = sdepth2.value,
                                        dist = sdist.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)
    
def on_depth_change2(change):
    global ew, ef, x, dx, V
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, 
                                        width1 = swidth1.value, depth1 = sdepth1.value,
                                        width2 = swidth2.value, depth2 = sdepth2.value,
                                        dist = sdist.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)
    
def on_dist_change(change):
    global ew, ef, x, dx, V
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, 
                                        width1 = swidth1.value, depth1 = sdepth1.value,
                                        width2 = swidth2.value, depth2 = sdepth2.value,
                                        dist = sdist.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)
    
def on_xfak_change(change):
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)

    plot_eigenfunktionen(ax, ew, ef, x, V, fak = sfak.value, single = ixx)

def on_press(event):
    global ann, ixx
    plot_eigenfunktionen(ax, ew, ef, x, V, fak = sfak.value, single = ixx)
    index_eig = sum(ew < event.ydata)
    
    ixx = index_eig - 1
    if ixx < 0:
        return 
    
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    
    try:
        ann.remove()
    except:
        pass
    
    ax[1].plot(x, V, c='r', linewidth=1.5)    
    ax[1].plot(x, sfak.value/100.0*abs(ef[:, ixx])**2+ew[ixx], linewidth=1.5, color=colors[ixx%len(colors)])
    ann = ax[1].annotate(s = 'n = ' + str(ixx+1), xy = (0, ew[ixx]), xytext = (-0.15, ew[ixx]), xycoords = 'data', color='k', size=15)

cid = fig.canvas.mpl_connect('button_press_event', on_press)

swidth1.observe(on_width_change1, names = 'value')
sdepth1.observe(on_depth_change1, names = 'value')
swidth2.observe(on_width_change2, names = 'value')
sdepth2.observe(on_depth_change2, names = 'value')
sdist.observe(on_dist_change, names = 'value')
sfak.observe(on_xfak_change, names = 'value')

display(HBox([swidth1, sdepth1, sdist]), HBox([swidth2, sdepth2, sfak]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This work has been done with the support of the EPFL Open Science Fund [OSSCAR](http://www.osscar.org).

<img src="http://www.osscar.org/wp-content/uploads/2019/03/OSSCAR-logo.png" style="height:40px; width: 200px"/>